# Object Detection

Hi 🙂, if you are seeing this notebook, you have succesfully started your first project on FloydHub 🚀, hooray!!

Recently the Chinese Police was on the first page of the most famous newspapers because they have succesfully detected a criminal in a pop concert with 60k people from security cameras using a facial recognition system. Here's a [link of the news](http://www.bbc.com/news/world-asia-china-43751276) if you missed it. This is not the first time that they used the same technology to catch criminals, but it is certainly quite amazing what this technology is able to achieve.

## "Where is Syd?"

In this project, we will use [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to detect Syd. It turns out that Syd participated to several Sport events of the last years, mostly during the opening ceremony. Unfortunately, we were able to get only some images of him during the ceremonies... but don't worry, Transfer Learning comes to help. We will use a pre-trained SSD model on COCO dataset to find Syd. If the trained model we will be good enough we will be able to detect Syd in the next games and tell to your family & friends: 'Look there, it's Syd! I found him ;)'.

![wanted Syd](https://raw.githubusercontent.com/floydhub/object-detection-template/master/images/wanted-syd.jpg)
*Are you able to catch Syd in this image?*

The project is structered around 3 notebooks:

- `p0_create_tf_record_dataset` where we build a TFRecords dataset from the images with bounding box annotations,
- `p1_training` where we perform the training, evaluation and model exportation,
- `p2_prediction` where we evaluate the model on new data.

**IMPORTANT**

Train on a GPU instance otherwise you will get an *Out-Of-Memory Error*.

## Part 1: Training a pretrained model with TF-Object-Detection-API repo

In this notebook, we will train an [SSD mobilnet v2](https://arxiv.org/pdf/1801.04381.pdf) model (pre-trained on the COCO object detection task) to spot Syd in the Olympics Opening Ceremony and stadiums.

For more info about Tensorflow Object Detection, please refer to [the official docs](https://github.com/tensorflow/models/tree/master/research/object_detection/g3doc).

### Initial Setup
Let's start by importing and installing the packages, setting the training variables and the path where to save the models.

In [ ]:
# Install extra-dependency
! pip -q install pycocotools

The current configuration is set to run on a single machine and without overwriting the `pipeline.config` file of the model.

In [1]:
import sys
sys.path.append("slim")

import functools
import json
import os
import tensorflow as tf

from object_detection import trainer
from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import dataset_util

from support import get_train_config, get_eval_config

# For reproducibility
from tensorflow import set_random_seed
from numpy.random import seed
seed(2018)
set_random_seed(2018)

Instructions for updating:
Use the retry module or similar alternatives.


## Train

The configuration is defined in the `PIPELINE_CONFING` file, edit this file or the `train_config` variable to modify the training settings. Training for 3700 steps will take about 30-35 minutes.

In [2]:
##################################################
# CONFIGURATION (Local, Multi-GPUs, Distributed) #
##################################################

MASTER = ''  # Name of the TensorFlow master to use
TASK = 0  # task id
NUM_CLONES = 1  # Number of clones to deploy per worker.

# Force clones to be deployed on CPU.  Note that even if 
# set to False (allowing ops to run on gpu), some ops may 
# still be run on the CPU if they have no GPU kernel.
CLONE_ON_CPU = False

WORKER_REPLICAS = 1  # Number of worker+trainer replicas.
PS_TASKS = 0  # Number of parameter server tasks. If None, does not use a parameter server.
TRAIN_DIR = 'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09'  # Directory to save the checkpoints and training summaries.

# Path to a pipeline_pb2.TrainEvalPipelineConfig config
# file. If provided, other configs are ignored
PIPELINE_CONFING_PATH = 'models/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config'
TRAIN_CONFIG_PATH = ''  # Path to a train_pb2.TrainConfig config file.
INPUT_CONFIG_PATH = ''  # Path to an input_reader_pb2.InputReader config file.
MODEL_CONFIG_PATH = ''  # Path to a model_pb2.DetectionModel config file.

# Get configuration for training
(create_input_dict_fn,
model_fn,
train_config,
master,
task,
worker_job_name,
ps_tasks,
worker_replicas,
is_chief,
graph_rewriter_fn) = get_train_config(TASK,
                                     PS_TASKS,
                                     TRAIN_DIR,
                                     PIPELINE_CONFING_PATH,
                                     TRAIN_CONFIG_PATH,
                                     MODEL_CONFIG_PATH,
                                     INPUT_CONFIG_PATH,
                                     WORKER_REPLICAS,
                                     MASTER)


######################################
# HOW TO EDIT TRAIN CONFIG FROM CODE #
######################################

# You can change the setting in this way,
# e.g. for batch size and num_steps
# train_config.batch_size = 4
# train_config.num_steps = 4000

print("Train configuration: \n", '-'*30, '\n', train_config)

Train configuration: 
 ------------------------------ 
 batch_size: 8
data_augmentation_options {
  random_horizontal_flip {
  }
}
data_augmentation_options {
  ssd_random_crop {
  }
}
optimizer {
  rms_prop_optimizer {
    learning_rate {
      exponential_decay_learning_rate {
        initial_learning_rate: 0.004000000189989805
        decay_steps: 10000
        decay_factor: 0.949999988079071
      }
    }
    momentum_optimizer_value: 0.8999999761581421
    decay: 0.8999999761581421
    epsilon: 1.0
  }
}
fine_tune_checkpoint: "/floyd/code/models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt"
num_steps: 3700
fine_tune_checkpoint_type: "detection"



In [3]:
# Training
trainer.train(
  create_input_dict_fn,
  model_fn,
  train_config,
  master,
  task,
  NUM_CLONES,
  worker_replicas,
  CLONE_ON_CPU,
  ps_tasks,
  worker_job_name,
  is_chief,
  TRAIN_DIR,
  graph_hook_fn=graph_rewriter_fn)

Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from /floyd/code/models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:

## Evaluate

Now it's time to evaluate our trained model.

**Note**

Unfortunately a bug prevent the correct output of [IOU(Intersection Over Union) metric](https://en.wikipedia.org/wiki/Jaccard_index). We will fix it as soon as possible.

In [4]:
from object_detection import evaluator
from object_detection.utils import label_map_util

tf.reset_default_graph()

##############################
# CONFIGURATION (Evaluation) #
##############################

# Directory to write eval summaries to.
EVAL_DIR = 'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/eval' 

# If training data should be evaluated for this job.
EVAL_TRAINING_DATA = False

# Option to only run a single pass of evaluation. 
# Overrides the `max_evals` parameter in the provided config
RUN_ONCE = True

# Directory containing checkpoints to evaluate, typically
# set to `train_dir` used in the training job.
CHECKPOINT_DIR = TRAIN_DIR

EVAL_CONFIG_PATH = ''  # Path to a eval_pb2.TrainConfig config file.
EVAL_INPUT_CONFIG_PATH = ''  # Path to an input_reader_pb2.InputReader config file.
MODEL_CONFIG_PATH = ''  # Path to a model_pb2.DetectionModel config file.

# Get configuration for Evaluation
(create_input_dict_fn,
model_fn,
eval_config,
categories,
graph_rewriter_fn) = get_eval_config(EVAL_DIR,
                                    PIPELINE_CONFING_PATH,
                                    EVAL_CONFIG_PATH,
                                    MODEL_CONFIG_PATH,
                                    EVAL_INPUT_CONFIG_PATH,
                                    EVAL_TRAINING_DATA,
                                    RUN_ONCE)

######################################
# HOW TO EDIT EVAL CONFIG FROM CODE #
######################################

# You can change the setting in this way,
# e.g. for evaluate only once
# eval_config.max_evals = 1

print("Eval configuration: \n", '-'*30, '\n', eval_config)

Eval configuration: 
 ------------------------------ 
 num_examples: 1
max_evals: 1



In [5]:
# Evaluate
evaluator.evaluate(
    create_input_dict_fn,
    model_fn,
    eval_config,
    categories,
    CHECKPOINT_DIR,
    EVAL_DIR,
    graph_hook_fn=graph_rewriter_fn)

INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:Restoring parameters from trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt-3700


INFO:tensorflow:Restoring parameters from trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt-3700


{'PascalBoxes_Precision/mAP@0.5IOU': 0.0,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/syd': 0.0,
 'Losses/Loss/localization_loss': 15.152876,
 'Losses/Loss/classification_loss': 24.296291}

## Exporting the model

In the last section of the notebook, we will export the model to run prediction on images converted into tensors (matrix representation for [height, width, color_channels]).

In [6]:
# Exporting the model for Evaluation
from google.protobuf import text_format
from object_detection import exporter
from object_detection.protos import pipeline_pb2

slim = tf.contrib.slim
tf.reset_default_graph()

config_override=''
input_shape=None
trained_checkpoint_prefix = os.path.join(TRAIN_DIR, 'model.ckpt-3700') # EDIT: model.ckpt-<iteration>
output_directory='trained_models/ssdlite_mobilenet_v2_coco_2018_05_09_exported'

# Type of input node. Can be one of :
# [`image_tensor`, `encoded_image_string_tensor`,`tf_example`]
input_type='image_tensor' 

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.gfile.GFile(PIPELINE_CONFING_PATH, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)
text_format.Merge(config_override, pipeline_config)
if input_shape:
    input_shape = [
        int(dim) if dim != '-1' else None
        for dim in input_shape.split(',')
    ]
else:
    input_shape = None
exporter.export_inference_graph(input_type, pipeline_config,
                                  trained_checkpoint_prefix,
                                  output_directory, input_shape)

INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


...

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: b'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09_exported/saved_model/saved_model.pb'


INFO:tensorflow:SavedModel written to: b'trained_models/ssdlite_mobilenet_v2_coco_2018_05_09_exported/saved_model/saved_model.pb'


Great! Will we be able to find Syd in never seen images? Let's jump to the  `p2_prediction` notebook to find the answer.